In [1]:
import glob
import os
DATA_DIR = "data/"


BASE_URL = 'http://localhost:8888'  #ur jupyter url

## 下載資料
我們使用政府資料開放平台的歷年交通事故資料。資料有分A1和A2類別，需要分別下載。

In [2]:
# ! wget -N -O {DATA_DIR}/traff_108_A1.csv https://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=6CF6661A-FA44-41DC-AF25-7BA9FF0F9652
# ! wget -N -O {DATA_DIR}/traff_108_A2_1.csv https://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=0F6EEADB-DF91-4B8B-A3EF-54173C6533C0
# ! wget -N -O {DATA_DIR}/traff_108_A2_2.csv https://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=701C543A-6D89-496D-9AB3-CAE2881788C5

# ! wget -N -O {DATA_DIR}/traff_109_A1.csv https://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=179FF667-2832-43D2-A4E4-26D5B031A186
# ! wget -N -O {DATA_DIR}/traff_109_A2_1.csv https://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=7CE87B70-68D5-4EAA-9F99-E3FF6E99CEA1
# ! wget -N -O {DATA_DIR}/traff_109_A2_2.csv https://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=649FB516-F037-4D9C-9FF7-9181D9AB22F5

# ! wget -N -O {DATA_DIR}/traff_110_A1.csv https://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=A9F35ABD-0826-4403-800D-D4ACDC1A151A
# ! wget -N -O {DATA_DIR}/traff_110_A2_1.csv https://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=58CC4AD0-A2A5-4F8E-A035-C32CAEF377E7
# ! wget -N -O {DATA_DIR}/traff_110_A2_2.csv https://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=AC183E22-B5A1-489E-AAD7-6A4B7795E61A

# ! wget -N -O {DATA_DIR}/traff_111_A1.csv https://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=BF1377E4-73DE-4DD1-9C2F-174E28EA1031
# ! wget -N -O {DATA_DIR}/traff_111_A2.csv https://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=219560DB-5617-4C5F-9754-8A4B96AC5F42

In [3]:
import cuxfilter
import cudf   #cuda supported pandas
import cugraph
from bokeh.models import NumeralTickFormatter
from bokeh.palettes import Inferno
from pathlib import Path
#from preprocess import *

## 載入資料及前處理

In [4]:
df = cudf.concat((cudf.read_csv(f) for f in glob.glob(os.path.join(DATA_DIR , "*.csv"))))

# Check
print('shape :', df.shape)
#print('columns :', data.columns)
print('null cells : ', df.isnull().sum().sum())
df.head()

shape : (1162908, 6)
null cells :  110


,﻿發生時間,發生地點,死亡受傷人數,車種,經度,緯度
0,110年01月01日 00時02分38秒,臺南市安平區中華西路2段口 / 臺南市安平區永華路2段口,死亡0;受傷1,自用-小客車;普通重型-機車,120.187485,22.989929
1,110年01月01日 00時05分00秒,臺東縣蘭嶼鄉東80線8.6公里,死亡0;受傷1,普通重型-機車,121.562208,22.054365
2,110年01月01日 00時05分00秒,臺中市西屯區上安里臺灣大道三段 / 臺中市西屯區上安里朝富路,死亡0;受傷2,大型重型1(550C.C.以上)-機車;普通重型-機車,120.639201,24.169277
3,110年01月01日 00時05分20秒,屏東縣三地門鄉台24線21公里,死亡0;受傷1,普通重型-機車;自用-小客車,120.638018,22.709899
4,110年01月01日 00時09分00秒,屏東縣新園鄉興安路與新南路口,死亡0;受傷1,普通重型-機車;自用-小客車,120.460589,22.534225


In [5]:
df.rename(columns = {'﻿發生時間':'發生時間'}, inplace = True)

In [6]:
print('rows null : ')
print(df[df.isnull().any(axis=1)].index)
df[df.isnull().any(axis=1)]

rows null : 
Int64Index([174079, 174080,    543,    544, 166485, 166486, 182336, 182337,
              1806,   1807, 187558, 187559, 173673, 173674,   1806,   1807,
              1814,   1815,  99757,  99758, 173029, 173030],
           dtype='int64')


,發生時間,發生地點,死亡受傷人數,車種,經度,緯度
174079,資料提供日期：111年03月01日,<NA>,<NA>,<NA>,<NA>,<NA>
174080,事故類別：A2類,<NA>,<NA>,<NA>,<NA>,<NA>
543,資料提供日期：111年05月02日,<NA>,<NA>,<NA>,<NA>,<NA>
544,事故類別：A1類,<NA>,<NA>,<NA>,<NA>,<NA>
166485,資料提供日期：109年07月01日,<NA>,<NA>,<NA>,<NA>,<NA>
166486,事故類別：A2類,<NA>,<NA>,<NA>,<NA>,<NA>
182336,資料提供日期：111年03月01日,<NA>,<NA>,<NA>,<NA>,<NA>
182337,事故類別：A2類,<NA>,<NA>,<NA>,<NA>,<NA>
1806,資料提供日期：110年07月01日,<NA>,<NA>,<NA>,<NA>,<NA>
1807,事故類別：A1類,<NA>,<NA>,<NA>,<NA>,<NA>


In [7]:
drop_ind = df[df.isnull().any(axis=1)].index
df=df.drop(index=drop_ind, errors=['ignore'])
print('null cells : ', df.isnull().sum().sum())

null cells :  0


In [8]:
from pyproj import Proj, Transformer
import re

def transform_coords(df, x='緯度', y='經度'):
    transform_4326_to_3857 = Transformer.from_crs('epsg:4326', 'epsg:3857')  #from GPS to Spherical Mercator for Google Map
    df['緯度'], df['經度'] = transform_4326_to_3857.transform(df[x].to_numpy(), df[y].to_numpy())
    return df

def transform_date(df, t='發生時間'):
    def trans_date(x):
        y, m, d, h=[],[],[],[]
        for n, i in enumerate(x):
            if (len(i)<20): i=x[n-1]
            i=re.split(r'[\u4e00-\u9fa5]',i)
            y.append(int(i[0]))
            m.append(int(i[1]))
            d.append(int(i[2]))
            try:
                h.append(int(i[3]))
            except:
                print('error : ',i)
        return y,m,d,h

    df['民國年'], df['月'], df['日'], df['時']= trans_date(df[t].to_numpy())
    return df

def transform_casualties(df, c='死亡受傷人數'):
    def trans_c(x):
        d,h=[],[]
        for i in x:
            i=re.sub(r'[\u4e00-\u9fa5]','',i)
            i=i.split(';')
            d.append(int(i[0]))
            h.append(int(i[1]))
        return d,h

    df['死亡'], df['受傷']= trans_c(df[c].to_numpy())
    return df

def transform_vehicleType(df, v='車種'):
    def trans_v(x):
        v=[]
        for i in x:
            i=i.split(';')
            v.append(i)
        return v

    df['車種'] = trans_v(df[v].to_numpy())
    return df

def transform_address(df, l='發生地點'):
    def trans_l(x):
        l1=[]
        l2=[]
        for i in x:
            l1.append(i[0:3])
            l2.append(i[3:6])
        return l1,l2

    df['縣市'], df['鄉鎮市區']=trans_l(df[l].to_numpy())
    return df


def preprocess(df):
    df = transform_date(df, t='發生時間')
    df = transform_coords(df, x='緯度', y='經度')  #from GPS to Spherical Mercator for Google Map、wikimap...
    df = transform_casualties(df, c='死亡受傷人數')
    df = transform_vehicleType(df, v='車種')
    df = transform_address(df, l='發生地點')
    df = df.drop(columns=['發生時間','發生地點','死亡受傷人數'], errors=['ignore'])
    return df


In [9]:
df = preprocess(df)
df.head()

,車種,經度,緯度,民國年,月,日,時,死亡,受傷,縣市,鄉鎮市區
0,"[自用-小客車, 普通重型-機車]",2.630801e+06,1.337921e+07,110,1,1,0,0,1,臺南市,安平區
1,[普通重型-機車],2.518054e+06,1.353224e+07,110,1,1,0,0,1,臺東縣,蘭嶼鄉
2,"[大型重型1(550C.C.以上)-機車, 普通重型-機車]",2.774049e+06,1.342949e+07,110,1,1,0,0,2,臺中市,西屯區
3,"[普通重型-機車, 自用-小客車]",2.596973e+06,1.342936e+07,110,1,1,0,0,1,屏東縣,三地門
4,"[普通重型-機車, 自用-小客車]",2.575787e+06,1.340961e+07,110,1,1,0,0,1,屏東縣,新園鄉


In [10]:
vehicle_count={}
for row in df['車種'].to_pandas():
    for key in row:
        if key in vehicle_count:
            vehicle_count[key]+=1
        else:
            vehicle_count[key]=1
vehicle_count = cudf.DataFrame(vehicle_count, index='數量').transpose().sort_values(by='數量', ascending=False)
vehicle_count['占比'] = (vehicle_count['數量'] / vehicle_count['數量'].sum()) * 100


In [11]:
vehicle_count[vehicle_count['占比']>1]

,數量,占比
普通重型-機車,1355080,54.804648
自用-小客車,572359,23.148400
乘客-人,162756,6.582479
自用-小貨車(含客、貨兩用),104219,4.215017
行人-人,59194,2.394033
腳踏自行車-慢車,45671,1.847111
計程車-小客車,35223,1.424554
普通輕型-機車,29524,1.194064


## 建立視覺化儀錶板

In [12]:
# add label map
years_map = {y:str(y)+'年' for y in df['民國年'].unique().to_pandas()}
years_map['']= 'All'

months_map = {m:str(m)+'月' for m in df['月'].unique().to_pandas()}
months_map['']= 'All'

In [13]:
cux_df = cuxfilter.DataFrame.from_dataframe(df)

In [14]:
layout = [
    [1, 1, 5],
    [1, 1, 6],
    [1, 1, 7],
    [1, 1, 0],
    [2, 3, 4],
    [2, 3, 4],
    ]

In [15]:
widget0 = cuxfilter.charts.number('死亡', aggregate_fn="sum", title="死亡人數", font_size='50pt')
widget1 = cuxfilter.charts.number('受傷', aggregate_fn="sum", title="受傷人數", font_size='50pt')
widget2 = cuxfilter.charts.number('受傷', aggregate_fn="count", title="案件數", font_size='50pt')

widget3 = cuxfilter.charts.multi_select('民國年', label_map=years_map)

widget4 = cuxfilter.charts.multi_select('月', label_map=months_map)


chart0 = cuxfilter.charts.scatter(
        x='緯度', y='經度', 
        tile_provider='OSM',
        point_size=6,
        pixel_shade_type='linear',
        pixel_spread='spread',
        title='案件位置分布圖'
      )

chart1 = cuxfilter.charts.bar('時', title='每小時案件數統計表')
chart1.y_axis_tick_formatter = NumeralTickFormatter(format="0")
chart2 = cuxfilter.charts.bar('月', title='每月案件數時統計表')
chart2.y_axis_tick_formatter = NumeralTickFormatter(format="0")
chart3 = cuxfilter.charts.bar('民國年', title='每年案件數統計表')
chart3.y_axis_tick_formatter = NumeralTickFormatter(format="0")



d = cux_df.dashboard([chart0, chart1, chart2, chart3, widget0, widget1, widget2], layout_array=layout, theme=cuxfilter.themes.rapids, title='台灣交通事故統計圖')

### 開啟儀錶板伺服器

In [16]:
d.show(notebook_url=BASE_URL, service_proxy='none')

Dashboard running at port 36421


Row(sizing_mode='stretch_both')
    [0] Button(button_type='success', name='open cuxfilter d..., sizing_mode='stretch_both')

### 關閉儀錶板伺服器

In [17]:
#d.stop()